In [12]:
import pandas as pd
import os

In [13]:
directory = '/Users/mrinaalarora/Downloads/Spotify Extended Streaming History'
dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.json'):
        filepath = os.path.join(directory, filename)
        df = pd.read_json(filepath)
        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_json('spotify_data.json', orient='records', lines=True)

In [16]:
my_data = pd.read_json('data/spotify_data.json',lines=True)
my_data

,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2023-04-01T00:28:08Z,31nr24ptwueq6pagibzbnwmugamm,ios,0,US,153.33.8.24,unknown,Do Not Disturb,Drake,More Life,...,None,None,None,fwdbtn,endplay,False,1.0,0.0,1680308888,False
1,2023-04-01T00:28:12Z,31nr24ptwueq6pagibzbnwmugamm,ios,812,US,153.33.8.24,unknown,Reminder,The Weeknd,Starboy,...,None,None,None,fwdbtn,endplay,False,1.0,0.0,1680308888,False
2,2023-04-01T00:28:13Z,31nr24ptwueq6pagibzbnwmugamm,ios,441,US,153.33.8.24,unknown,Keep The Family Close,Drake,Views,...,None,None,None,clickrow,fwdbtn,False,1.0,0.0,1680308892,False
3,2023-04-01T00:28:13Z,31nr24ptwueq6pagibzbnwmugamm,ios,0,US,153.33.8.24,unknown,9,Drake,Views,...,None,None,None,fwdbtn,fwdbtn,False,1.0,0.0,1680308893,False
4,2023-04-01T00:28:14Z,31nr24ptwueq6pagibzbnwmugamm,ios,0,US,153.33.8.24,unknown,With You,Drake,Views,...,None,None,None,fwdbtn,fwdbtn,False,1.0,0.0,1680308894,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102083,2023-10-12T02:16:19Z,31nr24ptwueq6pagibzbnwmugamm,ios,175720,US,172.56.209.32,unknown,HIGHEST IN THE ROOM,Travis Scott,HIGHEST IN THE ROOM,...,None,None,None,trackdone,trackdone,False,0.0,0.0,1697076804,False
102084,2023-10-12T02:19:35Z,31nr24ptwueq6pagibzbnwmugamm,ios,176640,US,172.56.209.32,unknown,A-Team,Travis Scott,A-Team,...,None,None,None,trackdone,trackdone,False,0.0,0.0,1697076980,False
102085,2023-10-12T02:26:06Z,31nr24ptwueq6pagibzbnwmugamm,ios,157157,US,172.56.209.32,unknown,THE SCOTTS,THE SCOTTS,THE SCOTTS,...,None,None,None,trackdone,endplay,False,1.0,0.0,1697077176,False
102086,2023-10-12T02:31:53Z,31nr24ptwueq6pagibzbnwmugamm,ios,347525,US,172.56.209.32,unknown,Somebody Else,The 1975,"I like it when you sleep, for you are so beaut...",...,None,None,None,playbtn,trackdone,True,0.0,0.0,1697077567,False
